In [ ]:
# https://pythonprogramming.net/crypto-rnn-model-deep-learning-python-tensorflow-keras/?completed=/balancing-rnn-data-deep-learning-python-tensorflow-keras/

import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
from sklearn import preprocessing
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

pd.options.display.max_columns = None

SEQ_LEN = 60 # how long in the past to look
FUTURE_PERIOD = 3 # how long in the future to predict
RATIO = "BCH-USD" # which currency to predict
ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]

EPOCHS = 20
BATCH_SIZE = 64
NAME = f'{RATIO}_{SEQ_LEN}_SEQ_{FUTURE_PERIOD}_PRED_{int(time.time())}'

In [ ]:
def classify(current, future): # to set target 0 or 1
  if float(future) > float(current):
    return 1
  else:
    return 0

def preprocess(df):
  df = df.drop("future", 1) # dont need future col after target is set

  for col in df.columns:
    if col != "target":
      df[col] = df[col].pct_change() # make everything a percntage, normalization
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values) # scaling
  df.dropna(inplace=True)

  sequential_data = []
  prev_days = deque(maxlen=SEQ_LEN)

  for i in df.values: # sequencing data per SEQ_LEN
    prev_days.append([n for n in i[:-1]])
    if len(prev_days) == SEQ_LEN:
      sequential_data.append([np.array(prev_days), i[-1]])

  random.shuffle(sequential_data)

  buys = []
  sells = []

  for seq, tar in sequential_data: # balancing sells/buys
    if tar == 0:
      sells.append([seq, tar])
    elif tar == 1:
      buys.append([seq, tar])

  random.shuffle(buys)
  random.shuffle(sells)

  lower = min(len(buys), len(sells))

  buys = buys[:lower]
  sells = sells[:lower]

  sequential_data = buys+sells
  random.shuffle(sequential_data)

  X = []
  y = []

  for seq, tar in sequential_data: # spliting data/labels -> x/y
    X.append(seq)
    y.append(tar)
  
  return np.array(X), y

In [ ]:
main_df = pd.DataFrame()

for ratio in ratios: # read files and create DF
  dataset = f'{ratio}.csv'
  df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
  df.rename(columns={"close": f'{ratio}_close', "volume":f'{ratio}_volume'}, inplace=True)
  df.set_index("time", inplace=True)
  df = df[[f'{ratio}_close', f'{ratio}_volume']]

  if len(main_df) == 0:
    main_df = df
  else:
    main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)
main_df.dropna(inplace=True)

main_df['future'] = main_df[f'{RATIO}_close'].shift(-FUTURE_PERIOD)
main_df['target'] = list(map(classify, main_df[f'{RATIO}_close'], main_df['future']))

times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05 * len(times))]

validation_main_df = main_df[main_df.index >= last_5pct] # split out of sample data for validation (last 5%)
main_df = main_df[(main_df.index < last_5pct)]

In [ ]:
train_x, train_y = preprocess(main_df)
validation_x, validation_y = preprocess(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"sells: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"validation:: sells: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train_y = np.asarray(train_y)
validation_y = np.asarray(validation_y)

train data: 25638 validation: 1290
sells: 12819, buys: 12819
validation:: sells: 645, buys: 645


In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'))

history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(validation_x, validation_y), callbacks=[tensorboard, checkpoint])

score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("models/{}".format(NAME))

Epoch 1/20
401/401 [==============================] - 29s 59ms/step - loss: 0.7912 - accuracy: 0.5076 - val_loss: 0.6961 - val_accuracy: 0.4977


INFO:tensorflow:Assets written to: models/RNN_Final-01-0.498.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-01-0.498.model/assets


Epoch 2/20
401/401 [==============================] - 20s 50ms/step - loss: 0.6981 - accuracy: 0.5245 - val_loss: 0.6938 - val_accuracy: 0.5465


INFO:tensorflow:Assets written to: models/RNN_Final-02-0.547.model/assets


...

INFO:tensorflow:Assets written to: models/RNN_Final-20-0.546.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-20-0.546.model/assets


Test loss: 0.8659203052520752
Test accuracy: 0.5457364320755005


INFO:tensorflow:Assets written to: models/BCH-USD_60_SEQ_3_PRED_1615550184/assets


INFO:tensorflow:Assets written to: models/BCH-USD_60_SEQ_3_PRED_1615550184/assets


In [1]:
# run this to see results in tensorboard inline
#%load_ext tensorboard
#%tensorboard --logdir logs/